In [ ]:
import   os
import   sys
sys.path.insert(0, os.path.abspath('.'))
from     ismrmrdUtils   import   rawMRutils
import   numpy          as       np
import   ismrmrd

## First - read in (fully sampled) EPI data from array coil

In [ ]:
dataHeader, dataArray, refDataArray = rawMRutils.returnHeaderAndData('./ScanArchive_EPI_converted.h5')

In [ ]:
# imageSpace = np.fft.fftshift(np.fft.fft2(dataArray, axes=(1,2)), axes=(1,2))

In [ ]:
# rawMRutils.computeAndPlot(imageSpace)

In [ ]:
dataArray.shape

In [ ]:
refDataArray.shape

In [ ]:
import matplotlib
import matplotlib.pyplot as mplt

In [ ]:
def computeN2GhostCorrection (angleData):          

   x = np.linspace(-0.5,0.5,len(angleData))
   w = np.abs(angleData)
   X = np.zeros([x.size,5])

   X[:,0] = w 
   X[:,1] = w*x
   X[:,2] = w*x**2
   X[:,3] = w*x**3
   X[:,4] = w*x**4

   B,resid,rank,sigma = np.linalg.lstsq(X,w*np.angle(angleData))

   oefit = B[0] + B[1]*x + B[2]*x**2 + B[3]*x**3 + B[4]*x**4

   return (np.exp (-1j * oefit))

In [ ]:
nCoils  = refDataArray.shape[0]
nNavs   = refDataArray.shape[2]
nSlices = refDataArray.shape[3]
nReps   = refDataArray.shape[4]

navData = np.zeros((nCoils, refDataArray.shape[1], nNavs), dtype=np.complex64)

imageSpace = np.fft.fftshift(np.fft.fft2(dataArray, axes=(1,2)), axes=(1,2))

# fig, ax = mplt.subplots(1, 1, sharex=True, sharey=True)

for s in range (nSlices):
   for r in range (nReps):
      for n in range (nNavs):
         if (n != 0): # First line does NOT contain reference data, so use to store
            navData[:, :, n] = np.fft.fftshift(np.fft.fft(np.fft.fftshift(refDataArray[:, :, n, s, r])))
            # ax.plot  (abs(navData[15, :, n]), label=str(n))
            # ax.legend()

      navData[:, :, 0] = (navData[:, :, 2] * (navData[:, :, 1] + navData[:, :, 3])*0.5)

      for c in range (nCoils):
         oddEvenPhase = computeN2GhostCorrection(navData[c, :, 0])
         # ax.plot (oddEvenPhase, label=str('phase')) 

         for p in range (dataArray.shape[2]):
            if (p % 2) == 0:
               imageSpace [c, :, p, s, r] = imageSpace [c, :, p, s, r] * oddEvenPhase

In [ ]:
# imageSpace = np.fft.fftshift(np.fft.fft2(dataArray, axes=(1,2)), axes=(1,2))
# rawMRutils.computeAndPlot(imageSpace)
plottedFigures = mplt.figure(figsize=(12,18))

subImages = plottedFigures.add_subplot(1, 1, (1))

subImages.imshow(np.sqrt(abs(imageSpace[15, :, :, 0, 0])), 'viridis')